In [45]:
import matplotlib.pylab as pl
import pandas as pd
import numpy as np
import zipfile
import os
import scipy.stats
%pylab inline

Populating the interactive namespace from numpy and matplotlib


**H0: there is no statistical difference in the trip duration distribution in the day and the evenings**
$$ \alpha = 0.05 $$

In [2]:
!curl https://s3.amazonaws.com/tripdata/201601-citibike-tripdata.zip?accessType=DOWNLOAD>201601-citibike-tripdata.zip
zip_ref = zipfile.ZipFile("201601-citibike-tripdata.zip", 'r')
zip_ref.extractall(os.getenv("PUIDATA"))
zip_ref.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 16.9M  100 16.9M    0     0  35.9M      0 --:--:-- --:--:-- --:--:-- 35.9M


In [3]:
!curl https://s3.amazonaws.com/tripdata/201607-citibike-tripdata.zip?accessType=DOWNLOAD>201607-citibike-tripdata.zip
zip_ref = zipfile.ZipFile("201607-citibike-tripdata.zip", 'r')
zip_ref.extractall(os.getenv("PUIDATA"))
zip_ref.close()

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.5M  100 46.5M    0     0  40.4M      0  0:00:01  0:00:01 --:--:-- 40.4M


In [6]:
df1 = pd.read_csv(os.getenv("PUIDATA") + "/" + "201601" + '-citibike-tripdata.csv')
df2 = pd.read_csv(os.getenv("PUIDATA") + "/" + "201607" + '-citibike-tripdata.csv')
df1.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,923,1/1/2016 00:00:41,1/1/2016 00:16:04,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22285,Subscriber,1958.0,1
1,379,1/1/2016 00:00:45,1/1/2016 00:07:04,476,E 31 St & 3 Ave,40.743943,-73.979661,498,Broadway & W 32 St,40.748549,-73.988084,17827,Subscriber,1969.0,1
2,589,1/1/2016 00:00:48,1/1/2016 00:10:37,489,10 Ave & W 28 St,40.750664,-74.001768,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,21997,Subscriber,1982.0,2
3,889,1/1/2016 00:01:06,1/1/2016 00:15:56,268,Howard St & Centre St,40.719105,-73.999733,3002,South End Ave & Liberty St,40.711512,-74.015756,22794,Subscriber,1961.0,2
4,1480,1/1/2016 00:01:12,1/1/2016 00:25:52,2006,Central Park S & 6 Ave,40.765909,-73.976342,2006,Central Park S & 6 Ave,40.765909,-73.976342,14562,Subscriber,1952.0,1


In [11]:
df1['starttime'].dropna(inplace = True)
df1['stoptime'].dropna(inplace = True)
df2['starttime'].dropna(inplace = True)
df2['stoptime'].dropna(inplace = True)

In [14]:
df1['starttime'] = pd.to_datetime(df1['starttime'])

In [15]:
df1['stoptime'] = pd.to_datetime(df1['stoptime'])

In [16]:
df2['starttime'] = pd.to_datetime(df1['starttime'])

In [17]:
df2['stoptime'] = pd.to_datetime(df2['stoptime'])

In [19]:
df1['data_month'] = 1
df2['data_month'] = 7
frames = [df1,df2]
df = pd.concat(frames)
df.reset_index(inplace = True)

In [28]:
df.columns

Index(['index', 'tripduration', 'starttime', 'stoptime', 'start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender', 'data_month'],
      dtype='object')

In [27]:
df_new = df.drop(['start station id',
       'start station name', 'start station latitude',
       'start station longitude', 'end station id', 'end station name',
       'end station latitude', 'end station longitude', 'bikeid', 'usertype',
       'birth year', 'gender'],axis=1)

In [43]:
df_morning = df_new[df_new['starttime'].dt.hour<12]
df_evening = df_new[df_new['starttime'].dt.hour>=12]

In [44]:
df_evening.head()

,index,tripduration,starttime,stoptime,data_month
3170,3170,1632,2016-01-01 12:00:00,2016-01-01 12:27:13,1
3171,3171,1201,2016-01-01 12:00:07,2016-01-01 12:20:08,1
3172,3172,1618,2016-01-01 12:00:12,2016-01-01 12:27:10,1
3173,3173,593,2016-01-01 12:00:16,2016-01-01 12:10:10,1
3174,3174,824,2016-01-01 12:00:21,2016-01-01 12:14:06,1


### KS test to compare the 2 samples

In [61]:
ks = scipy.stats.ks_2samp(df_morning['tripduration'], df_evening['tripduration'])
print(ks)

Ks_2sampResult(statistic=0.0080166027592110112, pvalue=2.5490820182388831e-13)


** Conclusion ** :The null hypothesis is that the two samples are the same. Since the pvalue is very small and less than 0.05 we can reject this hypothesis.

### Pearson's  test for correlation

In [59]:
df_morning_sorted = df_morning.sort(ascending= True, columns= ['tripduration'])
df_evening_sorted = df_evening.sort(ascending= True, columns= ['tripduration'])

/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:1: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  if __name__ == '__main__':
/usr/local/Anaconda3-5.0.0-Linux-x86_64/envs/PUI2016_Python3/lib/python3.5/site-packages/ipykernel/__main__.py:2: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  from ipykernel import kernelapp as app


In [63]:
ps = scipy.stats.pearsonr(df_morning_sorted['tripduration'], df_evening_sorted['tripduration'][:353576])
print(ps)

(0.081181239303994554, 0.0)


** Conclusion ** :The null hypothesis is that the two samples are uncorrelated. Since the pvalue is 0 we can reject this hypothesis.

### Spearman's  test for correlation

In [66]:
sc = scipy.stats.spearmanr(df_morning['tripduration'], df_evening['tripduration'][:353576])
print(sc)

SpearmanrResult(correlation=0.0076335105685535727, pvalue=5.6492765646193094e-06)


** Conclusion ** :The null hypothesis is that the two samples are uncorrelated. Since the pvalue is very small and lower than 0.05 we can reject this hypothesis.